In [268]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from catboost import CatBoostClassifier
import xgboost as xgb

In [269]:
data = pd.read_csv("BTC-USD_SIGNALS.csv")

In [270]:
data

,Date,Open,High,Low,Close,Volume,Variation,RSI,MACD,MACD_H,Confirmation Time,Transactions,Miners Revenue
0,2014-09-19,424.102997,427.834991,384.532013,394.795990,-0.792147,-1.859900,-2.943160,-0.057629,-0.015462,-0.917832,-2.189543,-0.817910
1,2014-09-20,394.673004,423.295990,389.882996,408.903992,-0.792200,0.871823,-2.943160,-0.059905,-0.018242,-0.997173,-2.189543,-0.817910
2,2014-09-21,408.084991,412.425995,393.181000,398.821014,-0.792718,-0.690803,-1.856302,-0.062400,-0.021104,-0.997173,-2.417949,-0.847329
3,2014-09-22,399.100006,406.915985,397.130005,402.152008,-0.792842,0.163318,-1.988871,-0.063948,-0.020636,-0.997173,-2.417949,-0.847329
4,2014-09-23,402.092010,441.557007,396.196991,435.790985,-0.791785,2.111516,-1.812541,-0.062275,-0.010871,-1.072907,-2.417949,-0.847329
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2923,2022-09-04,19832.470703,19999.689453,19636.816406,19986.712891,0.478227,0.148948,-1.113287,-0.844382,-0.254845,-1.202511,-0.040819,0.203364
2924,2022-09-05,19988.789063,20031.160156,19673.046875,19812.371094,0.658018,-0.278482,-1.133325,-0.831832,-0.166540,-0.875457,-0.040819,0.203364
2925,2022-09-06,19817.724609,20155.269531,18800.171875,18837.667969,1.393318,-1.325705,-1.548973,-0.892284,-0.308652,-0.875457,0.291152,0.153032
2926,2022-09-07,18837.683594,19427.171875,18644.466797,19290.324219,0.981876,0.568952,-1.180050,-0.893278,-0.249058,-0.875457,0.291152,0.153032


In [271]:
predict_data = data.copy().drop(["Open","Close","High","Low"],axis=1)
max_days = 20
target_range = 10
for i in range(1,max_days):#2jours
    predict_data[["Variation-{}".format(i),"Vol-{}".format(i),"RSI-{}".format(i),"MACD-{}".format(i),"MACD_H-{}".format(i),"CONF-{}".format(i),"TRANS-{}".format(i),"REV-{}".format(i)]] = data[["Variation","Volume","RSI","MACD","MACD_H","Confirmation Time","Transactions","Miners Revenue"]].shift(i)
    #predict_data[["Variation-{}".format(i),"Vol-{}".format(i),"RSI-{}".format(i),"MACD-{}".format(i),"MACD_H-{}".format(i)]] = data[["Variation","Volume","RSI","MACD","MACD_H"]].shift(i)
#predict_data["Target"] = (data["Variation"].shift(-1) >= 0)
predict_data["Target"] = (data["Close"].shift(-target_range) - data["Close"] >= 0)
predict_data["Target"] = np.where(predict_data["Target"] == True, 1, 0)
predict_data.dropna(inplace=True)
predict_data.reset_index(inplace=True,drop=True)
predict_data = predict_data[0:len(predict_data)-target_range]

c:\Users\Alexis\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\frame.py:3636: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self[k1] = value[k2]
<ipython-input-271-05f9f59bc36b>:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  predict_data["Target"] = (data["Close"].shift(-target_range) - data["Close"] >= 0)


In [272]:
predict_data = predict_data[[i % int(max_days / 3) == 0 for i in range(len(predict_data))]]

In [273]:
predict_data

,Date,Volume,Variation,RSI,MACD,MACD_H,Confirmation Time,Transactions,Miners Revenue,Variation-1,Vol-1,RSI-1,MACD-1,MACD_H-1,CONF-1,TRANS-1,REV-1,Variation-2,Vol-2,RSI-2,MACD-2,MACD_H-2,CONF-2,TRANS-2,REV-2,Variation-3,Vol-3,RSI-3,MACD-3,MACD_H-3,CONF-3,TRANS-3,REV-3,Variation-4,Vol-4,RSI-4,MACD-4,MACD_H-4,CONF-4,TRANS-4,...,Vol-15,RSI-15,MACD-15,MACD_H-15,CONF-15,TRANS-15,REV-15,Variation-16,Vol-16,RSI-16,MACD-16,MACD_H-16,CONF-16,TRANS-16,REV-16,Variation-17,Vol-17,RSI-17,MACD-17,MACD_H-17,CONF-17,TRANS-17,REV-17,Variation-18,Vol-18,RSI-18,MACD-18,MACD_H-18,CONF-18,TRANS-18,REV-18,Variation-19,Vol-19,RSI-19,MACD-19,MACD_H-19,CONF-19,TRANS-19,REV-19,Target
0,2014-10-08,-0.791299,1.236582,-1.323602,-0.077947,-0.008718,-0.786199,-2.180635,-0.877397,0.426006,-0.791578,-1.899452,-0.079434,-0.017265,-0.640141,-2.180635,-0.877397,0.719700,-0.790076,-1.300185,-0.079173,-0.021583,-0.640141,-2.180635,-0.877397,-0.710207,-0.789859,-1.478899,-0.077790,-0.022891,-0.640141,-2.218712,-0.875433,-2.258379,-0.791677,-1.476465,-0.074636,-0.018072,-0.678909,-2.218712,...,-0.791785,-1.812541,-0.062275,-0.010871,-1.072907,-2.417949,-0.847329,0.163318,-0.792842,-1.988871,-0.063948,-0.020636,-0.997173,-2.417949,-0.847329,-0.690803,-0.792718,-1.856302,-0.062400,-0.021104,-0.997173,-2.417949,-0.847329,0.871823,-0.792200,-2.943160,-0.059905,-0.018242,-0.997173,-2.189543,-0.817910,-1.859900,-0.792147,-2.943160,-0.057629,-0.015462,-0.917832,-2.189543,-0.817910,1
6,2014-10-14,-0.792118,0.640170,0.038393,-0.061421,0.032728,-0.563505,-2.369717,-0.870351,0.758195,-0.792283,0.059488,-0.065198,0.026201,-0.696039,-2.369717,-0.870351,1.107731,-0.793173,-0.158589,-0.068720,0.018969,-0.696039,-2.369717,-0.870351,-0.000044,-0.793385,-0.811570,-0.071720,0.011831,-0.696039,-2.118996,-0.873943,-0.298323,-0.791857,-0.887517,-0.073556,0.007151,-0.786199,-2.118996,...,-0.792420,-1.361295,-0.068238,-0.013923,-0.788904,-2.376687,-0.879626,-1.499524,-0.792868,-1.128547,-0.066427,-0.011757,-0.914226,-2.376687,-0.879626,-0.366595,-0.793300,-1.068725,-0.064062,-0.007029,-0.914226,-2.376687,-0.879626,-0.502217,-0.792976,-0.975839,-0.063161,-0.006456,-0.914226,-2.220623,-0.847062,-0.763885,-0.792706,-0.808468,-0.062336,-0.006014,-1.072907,-2.220623,-0.847062,0
12,2014-10-20,-0.793230,-0.497873,1.095716,-0.053655,0.023240,-0.693335,-2.331120,-0.860815,-0.178109,-0.793759,1.438512,-0.054010,0.026806,-0.706859,-2.331120,-0.860815,0.465292,-0.793482,1.174784,-0.055107,0.028562,-0.706859,-2.331120,-0.860815,0.028512,-0.793372,0.249057,-0.056636,0.029181,-0.706859,-2.203939,-0.863451,-0.853509,-0.792697,-0.063953,-0.057702,0.031644,-0.563505,-2.203939,...,-0.789859,-1.478899,-0.077790,-0.022891,-0.640141,-2.218712,-0.875433,-2.258379,-0.791677,-1.476465,-0.074636,-0.018072,-0.678909,-2.218712,-0.875433,-1.126181,-0.792500,-0.815338,-0.071125,-0.010750,-0.678909,-2.218712,-0.875433,-0.628994,-0.792960,-0.955663,-0.069521,-0.008545,-0.678909,-2.181195,-0.867599,-0.275389,-0.792736,-1.161418,-0.068810,-0.009043,-0.788904,-2.181195,-0.867599,0
18,2014-10-26,-0.793489,0.501026,-0.815530,-0.058690,-0.002273,-0.680712,-2.182495,-0.869972,-0.852377,-0.793144,-0.566571,-0.057883,-0.000852,-0.657271,-2.182495,-0.869972,-0.057982,-0.793272,-0.281475,-0.055974,0.004938,-0.657271,-2.182495,-0.869972,-1.723508,-0.792724,-0.367171,-0.054607,0.010203,-0.657271,-2.133794,-0.850907,-0.274856,-0.793471,0.651472,-0.052809,0.018353,-0.693335,-2.133794,...,-0.793385,-0.811570,-0.071720,0.011831,-0.696039,-2.118996,-0.873943,-0.298323,-0.791857,-0.887517,-0.073556,0.007151,-0.786199,-2.118996,-0.873943,0.833242,-0.789842,-0.924673,-0.075487,0.000955,-0.786199,-2.118996,-0.873943,1.236582,-0.791299,-1.323602,-0.077947,-0.008718,-0.786199,-2.180635,-0.877397,0.426006,-0.791578,-1.899452,-0.079434,-0.017265,-0.640141,-2.180635,-0.877397,0
24,2014-10-28,-0.793662,0.286520,-1.459895,-0.059422,-0.001825,-0.773577,-2.103547,-0.875014,0.286520,-0.793662,-1.459895,-0.059422,-0.001825,-0.680712,-2.103547,-0.875014,-

In [274]:
#corr = predict_data.corr("pearson")
#corr[["Target"]].to_clipboard()
#corr[["Target"]]

In [275]:
from sklearn.model_selection import train_test_split

In [367]:
split_mode = "RAND"

if split_mode == "RAND":
    train_data, test_data, train_labels, test_labels = train_test_split(predict_data.drop(["Date","Target"],axis=1), predict_data["Target"], test_size=0.33, random_state=1000)
elif split_mode == "STATIC":
    train_data = predict_data[0:int(0.66*len(predict_data))].drop(["Date","Target"],axis=1)
    train_labels = predict_data[0:int(0.66*len(predict_data))]["Target"]
    test_data = predict_data[int(0.66*len(predict_data)):len(predict_data)].drop(["Date","Target"],axis=1)
    test_labels = predict_data[int(0.66*len(predict_data)):len(predict_data)]["Target"]

In [331]:
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

In [413]:
algo = "CAT"

if algo == "MLP":
    model = MLPClassifier(hidden_layer_sizes=(200,2000,200),verbose=True)
elif algo == "RF":
    model = RandomForestClassifier(n_estimators=100,verbose=False)
elif algo == "CAT":
    model = CatBoostClassifier(iterations=100,depth=12)
elif algo == "TREE":
    model = DecisionTreeClassifier(splitter="random")
model.fit(train_data,train_labels)

Learning rate set to 0.052594
0:	learn: 0.6675813	total: 1.82s	remaining: 3m
1:	learn: 0.6450742	total: 3.65s	remaining: 2m 59s
2:	learn: 0.6277657	total: 5.5s	remaining: 2m 57s
3:	learn: 0.6107870	total: 7.48s	remaining: 2m 59s
4:	learn: 0.5894621	total: 9.9s	remaining: 3m 8s
5:	learn: 0.5721115	total: 11.9s	remaining: 3m 6s
6:	learn: 0.5545377	total: 13.9s	remaining: 3m 4s
7:	learn: 0.5344052	total: 15.9s	remaining: 3m 2s
8:	learn: 0.5183039	total: 17.8s	remaining: 2m 59s
9:	learn: 0.5058533	total: 19.7s	remaining: 2m 57s
10:	learn: 0.4911892	total: 21.7s	remaining: 2m 55s
11:	learn: 0.4757190	total: 23.5s	remaining: 2m 52s
12:	learn: 0.4598438	total: 25.4s	remaining: 2m 50s
13:	learn: 0.4455542	total: 27.3s	remaining: 2m 47s
14:	learn: 0.4318321	total: 29.3s	remaining: 2m 45s
15:	learn: 0.4187940	total: 31.3s	remaining: 2m 44s
16:	learn: 0.4051668	total: 33.2s	remaining: 2m 42s
17:	learn: 0.3931202	total: 35.1s	remaining: 2m 40s
18:	learn: 0.3823779	total: 37.1s	remaining: 2m 38s
19

In [414]:
model.score(test_data,test_labels)

0.6

In [415]:
model.score(train_data,train_labels)

1.0

In [416]:
from sklearn.metrics import confusion_matrix

In [417]:
preds = model.predict(test_data)

In [418]:
conf = confusion_matrix(test_labels,preds)
conf

array([[28, 35],
       [29, 68]], dtype=int64)

In [419]:
TN = conf[0,0] / (conf[0,0] + conf[1,0])
TN

0.49122807017543857

In [420]:
TP = conf[1,1] / (conf[1,1] + conf[0,1])
TP

0.6601941747572816